In [1]:
from transformers import TransfoXLTokenizer, TFTransfoXLForSequenceClassification
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('transfo-xl-wt103')
tokenizer.pad_token = tokenizer.eos_token

In [3]:
model = TFAutoModelForSequenceClassification.from_pretrained('transfo-xl-wt103',num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

Some layers from the model checkpoint at transfo-xl-wt103 were not used when initializing TFTransfoXLForSequenceClassification: ['crit']
- This IS expected if you are initializing TFTransfoXLForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFTransfoXLForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFTransfoXLForSequenceClassification were not initialized from the model checkpoint at transfo-xl-wt103 and are newly initialized: ['score']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Document","Sensitivity"]]
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)
train_x = np.array(train_x)
train_y = np.array(train_y)

tokenised_train_x = []

for elt in train_x:
    tokenised_train_x.append(tokenizer(elt,padding=True)['input_ids'])

In [5]:
print(tokenised_train_x[0])

[24, 24, 5661, 28991, 5790, 24, 24, 24, 24, 24, 74918, 190111, 101, 24, 2, 66142, 101, 190111, 2, 79780, 12, 24, 54011, 74918, 24, 115501, 24, 43, 3824, 101, 75, 24, 43, 24, 24, 24, 24, 38140, 24, 43, 77372, 24, 24, 159977, 24, 24, 15813, 24, 24, 222696, 43, 24, 16299, 65, 3, 22, 134039, 21, 21309, 3, 13, 42928, 5297, 11934, 3919, 22, 77372, 21, 54, 4002, 23366, 781, 26, 1, 4497, 1227, 7, 30837, 5179, 5, 54, 255, 1071, 6, 25320, 8, 9457, 12, 461, 3, 13, 42928, 287, 11934, 3919, 22, 212360, 21, 1929, 54, 44, 145, 17, 15031, 2, 18, 1227, 8852, 63, 1, 1007, 149, 2, 19, 67, 81, 299, 15037, 5, 239, 640, 3371, 1377, 3, 397, 6, 1, 8080, 3570, 4, 1, 3907, 22, 63197, 21, 5, 905, 2988, 2, 31, 63197, 2033, 1230, 11, 493, 20, 8660, 385, 5, 1, 1227, 54, 138, 710, 4972, 166, 92, 55, 93, 160, 3, 25888, 48, 450, 16, 30363, 24, 2, 799, 306, 27499, 23, 27, 1, 889, 4, 1, 1227, 2, 137, 922, 8, 46167, 1034, 15, 1, 15037, 3, 13, 63197, 54, 17170, 16, 1, 1395, 393, 1042, 10, 3397, 47, 1811, 3, 10, 3356, 2130

In [6]:
dataset = []
for i in range(len(train_x)):
    dataset.append({"label" : train_y[i], "text" : train_x[i]})


dataset = pd.DataFrame(dataset)
dataset = Dataset.from_pandas(dataset)

def preprocess_function(examples):
    return tokenizer(examples["text"],padding=False)

tokenised_dataset = dataset.map(preprocess_function, batched=True)

100%|██████████| 4/4 [00:38<00:00,  9.59s/ba]


In [7]:
tokenised_dataset

Dataset({
    features: ['input_ids', 'label', 'text'],
    num_rows: 3040
})

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="tf")

tf_train_dataset = tokenised_dataset.to_tf_dataset(
    columns=['input_ids', 'label','text'],
    label_cols=['label'],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
)

In [9]:
from transformers import create_optimizer

num_epochs = 4
batches_per_epoch = 3040 // 1
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)

In [10]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=[metric])

In [11]:
history=model.fit(tf_train_dataset,batch_size=1,epochs=4,verbose=1)

Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  41/3040 [..............................] - ETA: 28:04:16 - loss: 42.3178 - logits_loss: 1.7029 - loss_loss: 0.8013 - mems_1_loss: 4.8154 - mems_2_loss: 2.0742 - mems_3_loss: 1.9859 - mems_4_loss: 1.9693 - mems_5_loss: 1.9832 - mems_6_loss: 1.9767

ResourceExhaustedError:  OOM when allocating tensor with shape[4923,6523,1,16] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node tf_transfo_xl_for_sequence_classification/transformer/layers_._9/dec_attn/einsum_1/Einsum
 (defined at C:\Users\jack-\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\transfo_xl\modeling_tf_transfo_xl.py:223)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_46046]

Errors may have originated from an input operation.
Input Source operations connected to node tf_transfo_xl_for_sequence_classification/transformer/layers_._9/dec_attn/einsum_1/Einsum:
In[0] tf_transfo_xl_for_sequence_classification/transformer/layers_._9/dec_attn/add_1 (defined at C: